# Exercises: Linear dimensionality reduction 


Author: Stefano Pagani <stefano.pagani@polimi.it>.

Date: 2024

Course: Mathematical and numerical foundations of scientific machine learning

For further details on SVD, see Chapter 1, Brunton, S. L. and Kutz, J. N., Data-Driven Science and Engineering: Machine Learning, Dynamical Systems, and Control (2nd ed.).


In [ ]:
# imports

import numpy as np
import matplotlib.pyplot as plt
import cvxpy as cvx



Consider two different cases of reentrant activity (second one is commented).

In [ ]:
n=100
L=20

x=np.arange(-L,L,2*L/n)
y=x 

count_samples = 0

Xd=np.zeros((len(x)**2,10*n))
Td=np.zeros((len(x),len(x),10*n))
for ind_random in range(10):
    param = np.random.rand(1)
    print(param)
    for ind_sample in range(n):
        count = 0
        for ind_x in range(len(x)):
            for ind_y in range(len(y)):
                # case 1
                u = np.tanh(np.sqrt(x[ind_x]**2+y[ind_y]**2) * np.cos(np.angle(x[ind_x]+y[ind_y]*(0+(1.0+param)*1j))-(0.5)*(np.sqrt(x[ind_x]**2+y[ind_y]**2))+ ind_sample/10) )                
                # case 2
                #u = np.tanh(np.sqrt(x[ind_x]**2+y[ind_y]**2) * np.cos(np.angle(x[ind_x]+y[ind_y]*(0+(1.0+param)*1j))-(0.1+0.9*param)*(np.sqrt(x[ind_x]**2+y[ind_y]**2))+ ind_sample/10) )
                Xd[count,count_samples] = u
                count += 1

                Td[ind_x,ind_y,count_samples] = u
        
        count_samples += 1
            


Task 1: represent some snapshots of the reentrant activity at different timesteps. What is the effect of the parameter?

In [ ]:

plt.imshow( Td[:,:,250] )


Task 2: compute the SVD of the matrix and visualize the first basis functions.

In [ ]:

[U,S,V]=np.linalg.svd(Xd)

In [ ]:

fig,axs = plt.subplots(3,1)
axs = axs.reshape(-1)

axs[0].plot(range(1000),(S)/np.sum(S),'o')

# energy captured
axs[1].semilogy(range(1000),(S)/np.sum(S),'o')

# variance explained
axs[2].plot(range(1000),np.cumsum(S**2)/np.sum(S**2),'o')



In [ ]:
plt.imshow( np.reshape(U[:,0],(n,n) ) )


Test set

In [ ]:

count_samples = 0

Xd_test=np.zeros((len(x)**2,5*n))
Td_test=np.zeros((len(x),len(x),5*n))
for ind_random in range(5):
    param = np.random.rand(1)
    print(param)
    for ind_sample in range(n):
        count = 0
        for ind_x in range(len(x)):
            for ind_y in range(len(y)):
                # case 1
                u = np.tanh(np.sqrt(x[ind_x]**2+y[ind_y]**2) * np.cos(np.angle(x[ind_x]+y[ind_y]*(0+(1.0+param)*1j))-(0.5)*(np.sqrt(x[ind_x]**2+y[ind_y]**2))+ ind_sample/10) )                
                # case 2
                #u = np.tanh(np.sqrt(x[ind_x]**2+y[ind_y]**2) * np.cos(np.angle(x[ind_x]+y[ind_y]*(0+(1.0+param)*1j))-(0.1+0.9*param)*(np.sqrt(x[ind_x]**2+y[ind_y]**2))+ ind_sample/10) )
                
                Xd_test[count,count_samples] = u
                count += 1

                Td_test[ind_x,ind_y,count_samples] = u
        
        count_samples += 1

In [ ]:

u = Xd_test[:,55]


Task 4: solve the reconstruction problem using compressed sensing. Consider the case where $\Psi$ is composed by all the solutions stored in Xd and the case where $\Psi$ is composed by the first 500 columns of U.

In [ ]:
n_basis = 100

## Linear projection
Psi = U[:,:n_basis] 
urec = Psi @ ( Psi.T @ u )


In [ ]:
fig,axs = plt.subplots(1,3)


im0 = axs[0].imshow( np.reshape(u,(n,n) ) )
plt.colorbar(im0)

im1 = axs[1].imshow( np.reshape(urec,(n,n) ) )
plt.colorbar(im1)

im2 = axs[2].imshow( np.reshape(urec-u,(n,n) ) )
plt.colorbar(im2)


Task 6: represent the values of the coefficients obtained by projecting the first 100 test snapshots onto the low dimensional subspace.

In [ ]:
# coefficients
u_N = ( Psi.T @ Xd_test[:,:n] )

for i_N in range(n_basis):
    plt.plot(np.arange(0,n),u_N[i_N])